In [1]:
import numpy as np

In [2]:
class Subproblem_Quad_IP():
    def __init__(self, obj_coeff, constraints_coeff):
        self.n_dim = len(obj_coeff)
        self.obj_coeff = obj_coeff
        self.constraints_coeff = constraints_coeff
        
    def compute_obj(self, lamd):
        self.relaxed_obj_coeff = []
        for i in range(self.n_dim):
            self.relaxed_obj_coeff.append({'quad': obj_coeff[i], 'linear': (lamd * constraints_coeff[:,i]).sum()})
            
    def solve(self):
        self.opt_solution = np.zeros(self.n_dim)
        for i in range(self.n_dim):
            coeff = self.relaxed_obj_coeff[i]
            self.opt_solution[i] = round(-1 * coeff['linear']/ (2 *coeff['quad']))
        self.opt_solution = np.maximum(0, self.opt_solution)
    
    def solve_surrogate(self, dualproblem):
        self.compute_costfun(dualproblem)
        for i in range(self.n_dim):
            coeff = self.relaxed_obj_coeff[i]
            self.opt_solution[i] = round(-1 * coeff['linear'] / (2 *coeff['quad']))
            new_obj_value 
    
    def compute_costfun(self, dualproblem):
        costfun_orig = np.dot((self.opt_solution * self.opt_solution).T, obj_coeff)
        cost_constraints = 0
        for i in range(len(dualproblem.lamd)):
            cost_constraints += self.lamd[i] * (np.dot(self.opt_solution.T, self.constraints_coeff[i,:]) - rhs[i])
        return costfun_orig + cost_constraints
    
    def report(self, i):
        print("iteration times:", i)
        print("current solution:", self.opt_solution)

In [3]:
class Dual_Problem():
    def __init__(self, n_constraints):
        self.lamd = np.zeros(n_constraints)
        self.m, self.r, self.alpha = 10, 0.01, 1.0
        self.step_init, self.step, self.iteration_time = 0.0, 0.0, 0
        self.subgradients_init = np.zeros(n_constraints)
        self.subgradients = np.zeros(n_constraints)
        self.subg_norm_iterations, self.step_iterations, self.costfun_iterations = [], [], []
    
    def compute_subgradients(self, subproblem_qip):
        for i in range(len(self.lamd)):
            self.subgradients[i] = np.dot(subproblem_qip.opt_solution.T, subproblem_qip.constraints_coeff[i,:]) - rhs[i]
        if self.iteration_time == 0:
            self.subgradients_init = self.subgradients.copy()
                 
    def compute_stepsize(self, subproblem_qip):
        if self.iteration_time == 0:
            self.step_init = (417 - self.compute_costfun(subproblem_qip))/ np.linalg.norm(self.subgradients_init)**2
        else:
            p = 1 - 1/(self.iteration_time ** self.r)
            self.alpha = self.alpha * (1 - 1/(self.m * self.iteration_time ** p))
            if np.linalg.norm(self.subgradients) != 0:
                self.step = self.alpha * self.step_init * np.linalg.norm(self.subgradients_init) / np.linalg.norm(self.subgradients)
            else:
                self.step = 0
                print("subgradients = 0")
        self.step_iterations.append(self.step)
        
    def compute_stepsize_simple(self, subproblem_qip):
        self.step = (417 - self.compute_costfun(subproblem_qip))/ np.linalg.norm(self.subgradients)**2
        self.step_iterations.append(self.step)
        if self.iteration_time == 0:
            self.step_init = self.step
        
    def update_lamd(self):
        if self.iteration_time == 0:
            self.lamd = self.lamd + self.step_init * self.subgradients_init
        else:
            self.lamd = self.lamd + self.step * self.subgradients
        self.lamd = np.maximum(self.lamd, 0)
        self.iteration_time += 1
        
    def compute_costfun(self, subproblem):
        costfun_orig = np.dot((subproblem.opt_solution * subproblem.opt_solution).T, obj_coeff)
        cost_constraints = 0
        for i in range(len(self.lamd)):
            cost_constraints += self.lamd[i] * (np.dot(subproblem.opt_solution.T, subproblem.constraints_coeff[i,:]) - rhs[i])
        return costfun_orig + cost_constraints    
    
    def report(self, subproblem):
        print("subgradients:", self.subgradients)
        print("lamd: ", self.lamd)
        print("step:", self.step, "  dual problem cost function:", self.compute_costfun(subproblem))
        print("")


In [4]:
n_constraints = 2
lamd = np.zeros(n_constraints)
obj_coeff = np.array([0.5, 0.1, 0.5, 0.1, 0.5, 0.1])
constraints_coeff = np.array([[-1, 0.2, -1, 0.2, -1, 0.2], [-5, 1, -5, 1, -5, 1]])
rhs = [-48, -250]
subproblem = Subproblem_Quad_IP(obj_coeff, constraints_coeff)
subproblem.compute_obj(lamd)
print("the dimension of decision variables:", subproblem.n_dim)
print("coefficients of objective function:", subproblem.obj_coeff)
print("coefficients of constraints:", subproblem.constraints_coeff)
print("relaxed problem objective coefficients:", subproblem.relaxed_obj_coeff)
subproblem.solve()
print("optimal solution:", subproblem.opt_solution)

the dimension of decision variables: 6
coefficients of objective function: [0.5 0.1 0.5 0.1 0.5 0.1]
coefficients of constraints: [[-1.   0.2 -1.   0.2 -1.   0.2]
 [-5.   1.  -5.   1.  -5.   1. ]]
relaxed problem objective coefficients: [{'quad': 0.5, 'linear': 0.0}, {'quad': 0.1, 'linear': 0.0}, {'quad': 0.5, 'linear': 0.0}, {'quad': 0.1, 'linear': 0.0}, {'quad': 0.5, 'linear': 0.0}, {'quad': 0.1, 'linear': 0.0}]
optimal solution: [0. 0. 0. 0. 0. 0.]


In [5]:
dualproblem = Dual_Problem(n_constraints)
dualproblem.compute_subgradients(subproblem)
print("subgradients = ", dualproblem.subgradients_init)
dualproblem.compute_stepsize(subproblem)
print("step size = ", dualproblem.step)
print("the cost function of dual problem:", dualproblem.compute_costfun(subproblem))
dualproblem.update_lamd()
print("lamd = ", dualproblem.lamd)

subgradients =  [ 48. 250.]
step size =  0.0
the cost function of dual problem: 0.0
lamd =  [0.30886982 1.60869699]


In [6]:
max_itertimes = 30
for i in range(max_itertimes):
    subproblem.compute_obj(dualproblem.lamd)
    subproblem.solve()
    subproblem.report(i)
    dualproblem.compute_subgradients(subproblem)
    dualproblem.compute_stepsize(subproblem)
    dualproblem.update_lamd()
    dualproblem.report(subproblem)
print("optimal solution = ", subproblem.opt_solution)
print("subgradients = ", dualproblem.subgradients)

iteration times: 0
current solution: [8. 0. 8. 0. 8. 0.]
subgradients: [ 24. 130.]
lamd:  [0.57652021 3.05846994]
step: 0.011152099606251643   dual problem cost function: 507.4375776889173

iteration times: 1
current solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.57652021 4.38601905]
step: 0.13275491057053726   dual problem cost function: 427.860190485308

iteration times: 2
current solution: [23.  0. 23.  0. 23.  0.]
subgradients: [-21. -95.]
lamd:  [0.3182915  3.21784154]
step: 0.012296605353891887   dual problem cost function: 481.1209321881044

iteration times: 3
current solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.3182915 4.2968436]
step: 0.10790020647193525   dual problem cost function: 426.96843604630425

iteration times: 4
current solution: [22.  0. 22.  0. 22.  0.]
subgradients: [-18. -80.]
lamd:  [0.10452194 3.34675668]
step: 0.011876086557484162   dual problem cost function: 456.378070624652

iteration times: 5
current solution